In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
#from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

In [424]:
train = pd.read_csv('./train.csv')
train.drop(['종목명'],axis=1,inplace=True)
train['일자'] = pd.to_datetime(train['일자'], format='%Y%m%d')
#train = train[train['일자'].dt.year == 2023]
train = train[(train['일자'].dt.year == 2023) | (train['일자'].dt.year == 2022)]

train = train.reset_index(drop=True)
train

,일자,종목코드,거래량,시가,고가,저가,종가
0,2022-01-03,A060310,114207,3325,3370,3260,3300
1,2022-01-03,A095570,171188,5410,5470,5300,5320
2,2022-01-03,A006840,12856,19550,19600,19250,19550
3,2022-01-03,A054620,275806,14500,14750,13600,14000
4,2022-01-03,A265520,123779,25200,25250,24650,25150
...,...,...,...,...,...,...,...
693995,2023-05-30,A189980,272284,3005,3035,2955,2980
693996,2023-05-30,A000540,50218,3250,3255,3195,3215
693997,2023-05-30,A003280,130664,1344,1395,1340,1370
693998,2023-05-30,A037440,141932,9170,9260,9170,9200


In [425]:
dropped_samples = pd.DataFrame()

print(train['종목코드'].nunique())

stocks_to_drop = train[train['시가'] == 0]['종목코드'].unique()
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]

print(train['종목코드'].nunique())

code_counts = train['종목코드'].value_counts()

max_count = code_counts.max()

stocks_to_drop = code_counts[code_counts < max_count].index
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]

print(train['종목코드'].nunique())

train['종가변동률'] = train.groupby('종목코드')['종가'].pct_change()

stocks_to_drop = train[train['종가변동률'] <= -0.5]['종목코드'].unique()
dropped_samples = pd.concat([dropped_samples, train[train['종목코드'].isin(stocks_to_drop)]])

train = train[~train['종목코드'].isin(stocks_to_drop)]
train.drop(['종가변동률'],axis=1,inplace=True)

print(train['종목코드'].nunique())


2000
1840
1840
1812


In [428]:
train['종목코드'].head(40)

0     A060310
1     A095570
2     A006840
3     A054620
4     A265520
5     A211270
6     A027410
7     A282330
8     A126600
9     A138930
10    A001460
12    A001040
13    A079160
14    A035760
15    A311690
16    A000120
17    A011150
18    A097950
19    A051500
20    A058820
21    A023460
24    A000590
25    A012030
26    A016610
27    A005830
28    A000990
29    A139130
33    A000210
34    A001880
36    A068790
37    A007340
38    A004840
39    A241520
40    A065150
41    A155660
42    A069730
44    A017940
45    A245620
46    A037370
47    A050120
Name: 종목코드, dtype: object

In [399]:
print(dropped_samples['종목코드'].nunique())

188


In [400]:
train['일자'] = pd.to_datetime(train['일자'])  # 날짜 형식이 맞는지 확인
train = train.sort_values(by=['종목코드', '일자'])  # 주식 종목과 날짜 순으로 정렬

고정 = 
# 각 주식 종목에 대해 15일간의 종가를 구하기 위해 더 복잡한 접근법
for i in range(1, 16):
    train[f'종가_lag_{i}'] = train.groupby('종목코드')['종가'].shift(-i)

'''
for i in range(1, 15):
    train[f'종가_ratio_{i}'] = (train[f'종가_lag_{i}'] / train[f'종가_lag_{i+1}'] - 1)*100
'''
train

,일자,종목코드,거래량,시가,고가,저가,종가,종가_lag_1,종가_lag_2,종가_lag_3,...,종가_lag_6,종가_lag_7,종가_lag_8,종가_lag_9,종가_lag_10,종가_lag_11,종가_lag_12,종가_lag_13,종가_lag_14,종가_lag_15
502,2022-01-03,A000020,255857,14800,15000,14600,14950,14900.0,14550.0,14250.0,...,14350.0,14350.0,14100.0,13850.0,13350.0,13050.0,12600.0,12800.0,12800.0,12550.0
2502,2022-01-04,A000020,235805,15000,15000,14750,14900,14550.0,14250.0,14600.0,...,14350.0,14100.0,13850.0,13350.0,13050.0,12600.0,12800.0,12800.0,12550.0,11700.0
4502,2022-01-05,A000020,308187,14900,14900,14500,14550,14250.0,14600.0,14600.0,...,14100.0,13850.0,13350.0,13050.0,12600.0,12800.0,12800.0,12550.0,11700.0,11900.0
6502,2022-01-06,A000020,303389,14400,14600,14200,14250,14600.0,14600.0,14350.0,...,13850.0,13350.0,13050.0,12600.0,12800.0,12800.0,12550.0,11700.0,11900.0,11250.0
8502,2022-01-07,A000020,158132,14200,14650,14200,14600,14600.0,14350.0,14350.0,...,13350.0,13050.0,12600.0,12800.0,12800.0,12550.0,11700.0,11900.0,11250.0,11700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684145,2023-05-23,A383800,150364,8390,8390,8310,8330,8300.0,8310.0,8280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686145,2023-05-24,A383800,122457,8310,8340,8280,8300,8310.0,8280.0,8290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688145,2023-05-25,A383800,84241,8300,8310,8270,8310,8280.0,8290.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690145,2023-05-26,A383800,126681,8300,8310,8270,8280,8290.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [401]:
train['15일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=15).mean())
train['30일_지수이동평균'] = train.groupby('종목코드')['종가'].transform(lambda x: x.ewm(span=30).mean())
train['거래량_변화율'] = train.groupby('종목코드')['거래량'].pct_change()
train['15일_변동성'] = train.groupby('종목코드')['종가'].transform(lambda x: x.rolling(window=15, min_periods=1).std())
train

,일자,종목코드,거래량,시가,고가,저가,종가,종가_lag_1,종가_lag_2,종가_lag_3,...,종가_lag_10,종가_lag_11,종가_lag_12,종가_lag_13,종가_lag_14,종가_lag_15,15일_지수이동평균,30일_지수이동평균,거래량_변화율,15일_변동성
502,2022-01-03,A000020,255857,14800,15000,14600,14950,14900.0,14550.0,14250.0,...,13350.0,13050.0,12600.0,12800.0,12800.0,12550.0,14950.000000,14950.000000,NaN,NaN
2502,2022-01-04,A000020,235805,15000,15000,14750,14900,14550.0,14250.0,14600.0,...,13050.0,12600.0,12800.0,12800.0,12550.0,11700.0,14923.333333,14924.166667,-0.078372,35.355339
4502,2022-01-05,A000020,308187,14900,14900,14500,14550,14250.0,14600.0,14600.0,...,12600.0,12800.0,12800.0,12550.0,11700.0,11900.0,14781.952663,14791.040355,0.306957,217.944947
6502,2022-01-06,A000020,303389,14400,14600,14200,14250,14600.0,14600.0,14350.0,...,12800.0,12800.0,12550.0,11700.0,11900.0,11250.0,14621.268437,14641.964021,-0.015568,327.554067
8502,2022-01-07,A000020,158132,14200,14650,14200,14600,14600.0,14350.0,14350.0,...,12800.0,12550.0,11700.0,11900.0,11250.0,11700.0,14615.810413,14632.416191,-0.478781,285.043856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684145,2023-05-23,A383800,150364,8390,8390,8310,8330,8300.0,8310.0,8280.0,...,NaN,NaN,NaN,NaN,NaN,NaN,8426.377024,8477.441207,0.747321,111.128411
686145,2023-05-24,A383800,122457,8310,8340,8280,8300,8310.0,8280.0,8290.0,...,NaN,NaN,NaN,NaN,NaN,NaN,8410.579896,8465.993387,-0.185596,115.564865
688145,2023-05-25,A383800,84241,8300,8310,8270,8310,8280.0,8290.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8398.007409,8455.929298,-0.312077,115.255410
690145,2023-05-26,A383800,126681,8300,8310,8270,8280,8290.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8383.256483,8444.579020,0.503793,111.214079


In [402]:
#train['target'] = train['종가_lag_15'] / train['종가_lag_1']
train['target'] = train['종가_lag_15'] / train['종가']

In [403]:
def split_data(df, val_days=0, test_days=15):
    """
    데이터프레임을 주어진 일수에 따라 훈련, 검증, 테스트 세트로 분할합니다.

    Parameters:
    df (pandas.DataFrame): 분할할 데이터프레임
    val_days (int): 검증 세트로 사용할 일수
    test_days (int): 테스트 세트로 사용할 일수

    Returns:
    train (pandas.DataFrame): 훈련 세트
    val (pandas.DataFrame): 검증 세트
    test (pandas.DataFrame): 테스트 세트
    """
    df = df.sort_values(by=['종목코드', '일자'])

    df['rank'] = df.groupby('종목코드')['일자'].rank(method='first', ascending=False)

    train = df[df['rank'] > (val_days + test_days)]
    val = df[(df['rank'] > test_days) & (df['rank'] <= (val_days + test_days))]
    test = df[df['rank'] <= test_days]

    return train, val, test
train, _, test = split_data(train, val_days=0, test_days=15)

#train = train.dropna()
#test = test.dropna()

train = train.dropna()
train_x = train.drop(train.filter(regex='target').columns, axis=1)
train_x.drop(['rank'],axis=1,inplace=True)
test_x = test.drop(test.filter(regex='target').columns, axis=1)
test_x.drop(['rank'],axis=1,inplace=True)


train_y = train[train.filter(regex='target').columns]
train_y = pd.concat([train['종목코드'], train_y],axis=1)
test_y = test['종목코드']
train_y

,종목코드,target
2502,A000020,0.785235
4502,A000020,0.817869
6502,A000020,0.789474
8502,A000020,0.801370
10502,A000020,0.842466
...,...,...
654145,A383800,0.977700
656145,A383800,0.971897
658145,A383800,0.970794
660145,A383800,0.963912


In [404]:
train_x = train_x.drop(train_x.filter(regex='lag').columns, axis=1)
test_x = test_x.drop(test_x.filter(regex='lag').columns, axis=1)
test_x

,일자,종목코드,거래량,시가,고가,저가,종가,15일_지수이동평균,30일_지수이동평균,거래량_변화율,15일_변동성
664502,2023-05-09,A000020,77887,8600,8710,8520,8610,8513.783704,8545.408887,0.376266,103.071866
666502,2023-05-10,A000020,26578,8620,8650,8540,8600,8524.560741,8548.930894,-0.658762,91.573546
668502,2023-05-11,A000020,53660,8600,8640,8500,8500,8521.490648,8545.774062,1.018963,88.225901
670502,2023-05-12,A000020,50012,8500,8540,8370,8460,8513.804317,8540.240252,-0.067984,85.144135
672502,2023-05-15,A000020,35224,8410,8490,8350,8450,8505.828777,8534.418300,-0.295689,85.345237
...,...,...,...,...,...,...,...,...,...,...,...
684145,2023-05-23,A383800,150364,8390,8390,8310,8330,8426.377024,8477.441207,0.747321,111.128411
686145,2023-05-24,A383800,122457,8310,8340,8280,8300,8410.579896,8465.993387,-0.185596,115.564865
688145,2023-05-25,A383800,84241,8300,8310,8270,8310,8398.007409,8455.929298,-0.312077,115.255410
690145,2023-05-26,A383800,126681,8300,8310,8270,8280,8383.256483,8444.579020,0.503793,111.214079


In [405]:
# Linear Regression Model
from sklearn.linear_model import LinearRegression 
# to save model
import joblib
# use for standardization 
from scipy import stats

In [406]:
# Calculate Daily_Range and Mean
train_x['Daily_Range'] = train_x['종가'] - train_x['시가']
train_x['Mean'] = (train_x['고가']+train_x['저가']) / 2
#train_x['Mean'] = train_x['Mean'].astype(int)
'''
# Standardization 
train_x['시가'] = stats.zscore(train_x['시가'])
train_x['고가'] = stats.zscore(train_x['고가'])
train_x['저가'] = stats.zscore(train_x['저가'])
train_x['종가'] = stats.zscore(train_x['종가'])
train_x['거래량'] = stats.zscore(train_x['거래량'])
train_x['Daily_Range'] = stats.zscore(train_x['Daily_Range'])
train_x['Mean'] = stats.zscore(train_x['Mean'])
'''
# Calculate Daily_Range and Mean
test_x['Daily_Range'] = test_x['종가'] - test_x['시가']
test_x['Mean'] = (test_x['고가']+test_x['저가']) / 2
#test_x['Mean'] = test_x['Mean'].astype(int)
'''
# Standardization 
test_x['시가'] = stats.zscore(test_x['시가'])
test_x['고가'] = stats.zscore(test_x['고가'])
test_x['저가'] = stats.zscore(test_x['저가'])
test_x['종가'] = stats.zscore(test_x['종가'])
test_x['거래량'] = stats.zscore(test_x['거래량'])
test_x['Daily_Range'] = stats.zscore(test_x['Daily_Range'])
test_x['Mean'] = stats.zscore(test_x['Mean'])
'''
# 종목코드 별로 Standardization을 적용
features = ['시가', '고가', '저가', '종가', '거래량', 'Daily_Range', 'Mean']

for feature in features:
    train_x[feature] = train_x.groupby('종목코드')[feature].transform(stats.zscore)
    test_x[feature] = test_x.groupby('종목코드')[feature].transform(stats.zscore)


In [407]:
test_x

,일자,종목코드,거래량,시가,고가,저가,종가,15일_지수이동평균,30일_지수이동평균,거래량_변화율,15일_변동성,Daily_Range,Mean
664502,2023-05-09,A000020,-0.638475,-0.822145,-0.838691,-0.859448,-0.898771,8513.783704,8545.408887,0.376266,103.071866,-0.246959,-0.853425
666502,2023-05-10,A000020,-0.959192,-0.785332,-0.936592,-0.819349,-0.916795,8524.560741,8548.930894,-0.658762,91.573546,-0.389436,-0.889638
668502,2023-05-11,A000020,-0.789911,-0.822145,-0.952909,-0.899546,-1.097029,8521.490648,8545.774062,1.018963,88.225901,-0.769373,-0.934905
670502,2023-05-12,A000020,-0.812713,-1.006207,-1.116079,-1.160187,-1.169123,8513.804317,8540.240252,-0.067984,85.144135,-0.484420,-1.143131
672502,2023-05-15,A000020,-0.905149,-1.171863,-1.197663,-1.200286,-1.187147,8505.828777,8534.418300,-0.295689,85.345237,-0.104483,-1.206504
...,...,...,...,...,...,...,...,...,...,...,...,...,...
684145,2023-05-23,A383800,1.034524,-0.135565,-0.338951,-0.383762,-0.566139,8426.377024,8477.441207,0.747321,111.128411,-0.927401,-0.365531
686145,2023-05-24,A383800,0.302559,-0.875011,-0.801158,-0.697750,-0.905822,8410.579896,8465.993387,-0.185596,115.564865,0.384959,-0.764293
688145,2023-05-25,A383800,-0.699798,-0.967442,-1.078481,-0.802412,-0.792594,8398.007409,8455.929298,-0.312077,115.255410,0.909903,-0.963673
690145,2023-05-26,A383800,0.413349,-0.967442,-1.078481,-0.802412,-1.132277,8383.256483,8444.579020,0.503793,111.214079,0.122487,-0.963673


In [408]:
test_x = test_x[test_x['일자'] == '2023-05-30']
test_x

,일자,종목코드,거래량,시가,고가,저가,종가,15일_지수이동평균,30일_지수이동평균,거래량_변화율,15일_변동성,Daily_Range,Mean
692502,2023-05-30,A000020,0.133326,1.681102,1.331462,1.386076,1.065789,9273.198942,8993.470784,0.026007,574.305378,-1.529247,1.364635
692118,2023-05-30,A000040,-0.551675,0.267400,-0.335766,-0.204162,-0.733317,691.972246,666.916446,-0.402565,20.796520,-1.398442,-0.283214
692243,2023-05-30,A000050,2.621497,-0.383032,-0.816646,-0.102424,0.297200,10438.157465,10506.329663,1.757588,65.012819,0.501717,-0.665566
692760,2023-05-30,A000070,-0.635578,-1.463468,-1.422481,-0.656864,-1.639449,74081.702527,74433.918930,-0.600414,921.799482,-0.231749,-1.306354
693803,2023-05-30,A000080,0.261929,0.698164,0.000000,-0.714742,-1.108103,23280.945511,23042.647487,0.542465,354.964787,-1.472658,-0.391889
...,...,...,...,...,...,...,...,...,...,...,...,...,...
692702,2023-05-30,A369370,0.536573,-0.678462,-1.018297,-0.448013,-1.482518,1956.538082,1980.997963,-0.290558,23.645799,-0.536219,-0.780341
693805,2023-05-30,A373200,0.770834,-1.540889,-1.216890,-1.147513,-1.575248,4846.641778,5186.919116,0.249867,122.659032,0.087939,-1.209025
693955,2023-05-30,A378850,-0.742009,-0.200455,-0.631511,-0.693232,-0.780570,4545.414724,4430.279689,-0.074449,166.201971,-0.707997,-0.714392
693129,2023-05-30,A383310,-0.473280,-0.664831,-0.995326,-0.380457,-1.013336,63960.086591,65496.077894,-0.755145,1144.052946,-0.119665,-0.972209


In [409]:
pred_df = test_x[['종목코드']]
pred_df

,종목코드
692502,A000020
692118,A000040
692243,A000050
692760,A000070
693803,A000080
...,...
692702,A369370
693805,A373200
693955,A378850
693129,A383310


In [410]:
test_x

,일자,종목코드,거래량,시가,고가,저가,종가,15일_지수이동평균,30일_지수이동평균,거래량_변화율,15일_변동성,Daily_Range,Mean
692502,2023-05-30,A000020,0.133326,1.681102,1.331462,1.386076,1.065789,9273.198942,8993.470784,0.026007,574.305378,-1.529247,1.364635
692118,2023-05-30,A000040,-0.551675,0.267400,-0.335766,-0.204162,-0.733317,691.972246,666.916446,-0.402565,20.796520,-1.398442,-0.283214
692243,2023-05-30,A000050,2.621497,-0.383032,-0.816646,-0.102424,0.297200,10438.157465,10506.329663,1.757588,65.012819,0.501717,-0.665566
692760,2023-05-30,A000070,-0.635578,-1.463468,-1.422481,-0.656864,-1.639449,74081.702527,74433.918930,-0.600414,921.799482,-0.231749,-1.306354
693803,2023-05-30,A000080,0.261929,0.698164,0.000000,-0.714742,-1.108103,23280.945511,23042.647487,0.542465,354.964787,-1.472658,-0.391889
...,...,...,...,...,...,...,...,...,...,...,...,...,...
692702,2023-05-30,A369370,0.536573,-0.678462,-1.018297,-0.448013,-1.482518,1956.538082,1980.997963,-0.290558,23.645799,-0.536219,-0.780341
693805,2023-05-30,A373200,0.770834,-1.540889,-1.216890,-1.147513,-1.575248,4846.641778,5186.919116,0.249867,122.659032,0.087939,-1.209025
693955,2023-05-30,A378850,-0.742009,-0.200455,-0.631511,-0.693232,-0.780570,4545.414724,4430.279689,-0.074449,166.201971,-0.707997,-0.714392
693129,2023-05-30,A383310,-0.473280,-0.664831,-0.995326,-0.380457,-1.013336,63960.086591,65496.077894,-0.755145,1144.052946,-0.119665,-0.972209


In [411]:
train_x.drop(['일자','종목코드'],axis=1,inplace=True)
test_x.drop(['일자','종목코드'],axis=1,inplace=True)
train_x.drop(['시가','고가', '저가'],axis=1,inplace=True)
test_x.drop(['시가','고가', '저가'],axis=1,inplace=True)
test_x

,거래량,종가,15일_지수이동평균,30일_지수이동평균,거래량_변화율,15일_변동성,Daily_Range,Mean
692502,0.133326,1.065789,9273.198942,8993.470784,0.026007,574.305378,-1.529247,1.364635
692118,-0.551675,-0.733317,691.972246,666.916446,-0.402565,20.796520,-1.398442,-0.283214
692243,2.621497,0.297200,10438.157465,10506.329663,1.757588,65.012819,0.501717,-0.665566
692760,-0.635578,-1.639449,74081.702527,74433.918930,-0.600414,921.799482,-0.231749,-1.306354
693803,0.261929,-1.108103,23280.945511,23042.647487,0.542465,354.964787,-1.472658,-0.391889
...,...,...,...,...,...,...,...,...
692702,0.536573,-1.482518,1956.538082,1980.997963,-0.290558,23.645799,-0.536219,-0.780341
693805,0.770834,-1.575248,4846.641778,5186.919116,0.249867,122.659032,0.087939,-1.209025
693955,-0.742009,-0.780570,4545.414724,4430.279689,-0.074449,166.201971,-0.707997,-0.714392
693129,-0.473280,-1.013336,63960.086591,65496.077894,-0.755145,1144.052946,-0.119665,-0.972209


In [412]:
train_x = train_x.reset_index(drop = True)
train_y = train_y.reset_index(drop = True)
test_x = test_x.reset_index(drop = True)

In [413]:
# Instantiation and Learning
model = LinearRegression()
model.fit(train_x, train_y['target'])

LinearRegression()

In [414]:
y_pred = model.predict(test_x)
y_pred

array([0.95815199, 1.029392  , 0.97306534, ..., 1.027771  , 1.03309921,
       1.03741158])

In [415]:
pred_df['rate'] = y_pred
pred_df

,종목코드,rate
692502,A000020,0.958152
692118,A000040,1.029392
692243,A000050,0.973065
692760,A000070,1.063018
693803,A000080,1.047141
...,...,...
692702,A369370,1.062444
693805,A373200,1.062042
693955,A378850,1.027771
693129,A383310,1.033099


In [416]:
pred_df.sort_values('rate')

,종목코드,rate
693454,A033320,0.851556
693189,A045060,0.857188
693149,A069410,0.859523
692548,A171120,0.860601
693619,A031820,0.860636
...,...,...
692757,A225190,1.100131
692091,A096760,1.102966
693204,A226400,1.112295
692978,A002790,1.113213


In [60]:
pred_df.to_csv('lr_back_up.csv',index=False)

# XGB

In [259]:
train_y

,종목코드,target
0,A000020,0.839465
1,A000020,0.785235
2,A000020,0.817869
3,A000020,0.789474
4,A000020,0.801370
...,...,...
601579,A383800,0.977700
601580,A383800,0.971897
601581,A383800,0.970794
601582,A383800,0.963912


In [261]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

model = XGBRegressor(tree_method='gpu_hist', gpu_id=0)
xgb_model = model.fit(train_x, train_y['target'])
y_pred = xgb_model.predict(test_x)
y_pred

array([0.96989125, 1.0270995 , 1.041522  , ..., 1.0302619 , 1.0787652 ,
       1.0139894 ], dtype=float32)

In [262]:
pred_df['rate'] = y_pred
pred_df

,종목코드,rate
692502,A000020,0.969891
692118,A000040,1.027099
692243,A000050,1.041522
692760,A000070,1.066456
693803,A000080,1.095633
...,...,...
692702,A369370,1.064545
693805,A373200,1.051412
693955,A378850,1.030262
693129,A383310,1.078765


# Gluon

In [283]:
#AutoGluon 쓸때는 numpy 1.23으로
#pycaret 쓸때는 numpy 1.20으로

import os
import torch
import random
import numpy as np
import pandas as pd

from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate, train_test_split

from sklearn.model_selection import train_test_split

#######################Modeling########################


train = pd.concat([train_x,train_y['target']], axis = 1)

train_data = TabularDataset(train)
test_data = TabularDataset(test_x)




print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")
print("==================Tabular_complete========================")


predictor = TabularPredictor(label='target',  eval_metric='root_mean_squared_error').fit(train_data, presets='best_quality',  ag_args_fit={'num_gpus': 0})


print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")
print("==================learning_complete========================")



y_pred = predictor.predict(test_data)



#y_pred = pd.DataFrame(y_pred, columns=['rate'])
pred_df['rate'] = y_pred
pred_df

No path specified. Models will be saved in: "AutogluonModels\ag-20230712_023227\"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
	Consider setting `time_limit` to ensure training finishes within an expected duration or experiment with a small portion of `train_data` to identify an ideal `presets` and `hyperparameters` configuration.
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230712_023227\"
AutoGluon Version:  0.8.2
Python Version:     3.8.16
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   100.41 GB / 511.09 GB (19.6%)
Train Data Rows:    601584
Train Data Columns: 7


Label Column: target
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (4.397435897435898, 0.16398330351818724, 0.9971, 0.12219)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    14002.8 MB


==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================
==================Tabular_complete========================


	Train Data (Original)  Memory Usage: 33.69 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 7 | ['거래량', '시가', '고가', '저가', '종가', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 7 | ['거래량', '시가', '고가', '저가', '종가', ...]
	0.9s = Fit runtime
	7 features in original data used to generate 7 features in processed data.
	Train Data (Processed) Memory Usage: 33.69 MB (0.2% of available memory)
Data preprocessing and feature engineering runtime = 0.98s ...
AutoGluo

==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================
==================learning_complete========================


,종목코드,rate
692502,A000020,NaN
692118,A000040,NaN
692243,A000050,NaN
692760,A000070,NaN
693803,A000080,NaN
...,...,...
692702,A369370,NaN
693805,A373200,NaN
693955,A378850,NaN
693129,A383310,NaN


In [287]:
pred_df['rate'] = y_pred.values
pred_df

,종목코드,rate
692502,A000020,0.977100
692118,A000040,1.026836
692243,A000050,1.020228
692760,A000070,1.035867
693803,A000080,1.030727
...,...,...
692702,A369370,1.027884
693805,A373200,1.068503
693955,A378850,1.039543
693129,A383310,1.030502


# Metric

In [417]:
answer_df = pd.read_csv('Answer_real.csv', encoding = "CP949")
answer_df = answer_df[['종목코드','등락률']]
answer_df = answer_df.sort_values('종목코드')
# '종목코드' 피처에서 맨 앞 글자 제거
#answer_df['종목코드'] = answer_df['종목코드'].str[1:]
answer_df


,종목코드,등락률
334,A000020,8.04
1861,A000040,-12.17
1358,A000050,-3.76
764,A000070,0.97
1256,A000080,-2.89
...,...,...
1077,A375500,-1.24
1355,A378850,-3.74
1294,A383220,-3.20
683,A383310,1.77


In [418]:
#prediction_df = pd.read_csv('submission.csv')
#prediction_df['등락률'] = np.where(prediction_df['순위'] <= 1000, 1, -1)
#prediction_df

In [419]:
prediction_df = pred_df[['종목코드','rate']]
prediction_df.rename(columns={'rate': '등락률'}, inplace=True)
prediction_df

,종목코드,등락률
692502,A000020,0.958152
692118,A000040,1.029392
692243,A000050,0.973065
692760,A000070,1.063018
693803,A000080,1.047141
...,...,...
692702,A369370,1.062444
693805,A373200,1.062042
693955,A378850,1.027771
693129,A383310,1.033099


In [420]:
prediction_df = prediction_df.reset_index(drop = True)
prediction_df

,종목코드,등락률
0,A000020,0.958152
1,A000040,1.029392
2,A000050,0.973065
3,A000070,1.063018
4,A000080,1.047141
...,...,...
1807,A369370,1.062444
1808,A373200,1.062042
1809,A378850,1.027771
1810,A383310,1.033099


In [421]:
answer_df = answer_df[answer_df['종목코드'].isin(prediction_df['종목코드'])]
answer_df = answer_df.reset_index(drop = True)
answer_df

,종목코드,등락률
0,A000020,8.04
1,A000040,-12.17
2,A000050,-3.76
3,A000070,0.97
4,A000080,-2.89
...,...,...
1807,A369370,-9.10
1808,A373200,16.14
1809,A378850,-3.74
1810,A383310,1.77


In [422]:
from sklearn.metrics import accuracy_score, mean_absolute_error

# 정답 데이터프레임과 예측 결과 데이터프레임은 pandas DataFrame 형태라고 가정합니다.
# 등락률은 실수 형태의 값이라고 가정하고, 부호에 따라 'up' 또는 'down'으로 분류합니다.

# 등락률의 부호에 따른 분류 결과를 새로운 열로 추가합니다.
answer_df['trend'] = answer_df['등락률'].apply(lambda x: 'up' if x > 0 else 'down')
prediction_df['trend'] = prediction_df['등락률'].apply(lambda x: 'up' if x > 0 else 'down')

# 1. 전체 종목에 대한 Accuracy
total_accuracy = accuracy_score(answer_df['trend'], prediction_df['trend'])

# 변동률이 높은 상위 200개 및 하위 200개 종목 추출
top200_df = answer_df.sort_values(by='등락률', ascending=False).head(200)
bottom200_df = answer_df.sort_values(by='등락률', ascending=True).head(200)

# 2. 상위 200개 종목 및 하위 200개 종목에 대한 Accuracy
top200_accuracy = accuracy_score(top200_df['trend'], prediction_df.loc[top200_df.index, 'trend'])
bottom200_accuracy = accuracy_score(bottom200_df['trend'], prediction_df.loc[bottom200_df.index, 'trend'])

# 3. 상위 200개 종목 및 하위 200개 종목에 대한 MAE
# 이 경우 '등락률'은 실제 변동률을 예측한 연속적인 값이라고 가정합니다.
top200_mae = mean_absolute_error(top200_df['등락률'], prediction_df.loc[top200_df.index, '등락률'])
bottom200_mae = mean_absolute_error(bottom200_df['등락률'], prediction_df.loc[bottom200_df.index, '등락률'])

print("Total Accuracy: ", total_accuracy)
print("Top 200 Accuracy: ", top200_accuracy)
print("Bottom 200 Accuracy: ", bottom200_accuracy)
print("Top 200 MAE: ", top200_mae)
print("Bottom 200 MAE: ", bottom200_mae)


Total Accuracy:  0.456401766004415
Top 200 Accuracy:  1.0
Bottom 200 Accuracy:  0.0
Top 200 MAE:  26.9968081769791
Bottom 200 MAE:  15.704816521294957


In [423]:
from sklearn.metrics import accuracy_score, mean_absolute_error

# 정답 데이터프레임과 예측 결과 데이터프레임은 pandas DataFrame 형태라고 가정합니다.
# 등락률은 실수 형태의 값이라고 가정하고, 부호에 따라 'up' 또는 'down'으로 분류합니다.

# 등락률의 부호에 따른 분류 결과를 새로운 열로 추가합니다.
answer_df['trend'] = answer_df['등락률'].apply(lambda x: 'up' if x > 0 else 'down')
prediction_df['trend'] = prediction_df['등락률'].apply(lambda x: 'up' if x > 1 else 'down')

# 1. 전체 종목에 대한 Accuracy
total_accuracy = accuracy_score(answer_df['trend'], prediction_df['trend'])

# 변동률이 높은 상위 200개 및 하위 200개 종목 추출
top200_df = answer_df.nlargest(200, '등락률')
bottom200_df = answer_df.nsmallest(200, '등락률')

# 2. 상위 200개 종목 및 하위 200개 종목에 대한 Accuracy
top200_accuracy = accuracy_score(top200_df['trend'], prediction_df.loc[top200_df.index, 'trend'])
bottom200_accuracy = accuracy_score(bottom200_df['trend'], prediction_df.loc[bottom200_df.index, 'trend'])

# 3. 상위 200개 종목 및 하위 200개 종목에 대한 MAE
# 이 경우 '등락률'은 실제 변동률을 예측한 연속적인 값이라고 가정합니다.
top200_mae = mean_absolute_error(top200_df['등락률'], prediction_df.loc[top200_df.index, '등락률'])
bottom200_mae = mean_absolute_error(bottom200_df['등락률'], prediction_df.loc[bottom200_df.index, '등락률'])

print("Total Accuracy: ", total_accuracy)
print("Top 200 Accuracy: ", top200_accuracy)
print("Bottom 200 Accuracy: ", bottom200_accuracy)
print("Top 200 MAE: ", top200_mae)
print("Bottom 200 MAE: ", bottom200_mae)


Total Accuracy:  0.4663355408388521
Top 200 Accuracy:  0.41
Bottom 200 Accuracy:  0.515
Top 200 MAE:  26.9968081769791
Bottom 200 MAE:  15.704816521294957


# Sub

In [ ]:
sample_submission = pd.read_csv('./sample_submission.csv')
sample_submission

In [ ]:
baseline_submission = sample_submission[['종목코드']].merge(results_df[['종목코드', '순위']], on='종목코드', how='left')
baseline_submission